In [1]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, evaluate, dump
from surprise import SVD, SVDpp, KNNBaseline, KNNWithMeans, CoClustering
import threading
import thread


In [2]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)


In [3]:
def processDataFile(data,counter):
    df_nan = pd.DataFrame(pd.isnull(data.Rating))
    df_nan = df_nan[df_nan['Rating'] == True]
    df_nan = df_nan.reset_index()
    # print df_na
    movie_np = []
    movie_id = 1
    # print df_nan['index'][1:],df_nan['index'][:-1]
    a = zip(df_nan['index'][1:],df_nan['index'][:-1])
    store_df = pd.DataFrame([])
    temp_total_array = np.zeros(((0),1))
    for i,j in a:
        temp_np_array = np.full((i-j,1), counter)
        temp_total_array =  np.concatenate((temp_total_array, temp_np_array))
        counter = counter+1
        #print counter
    #print temp_total_array.shape
    remaining =  data.shape[0]- temp_total_array.shape[0]
    temp_np_array = np.full((remaining,1), counter)
    temp_total_array =  np.concatenate((temp_total_array, temp_np_array))
    data['movie_id'] = temp_total_array
    data= data.dropna(thresh =3)
    return (data,counter)

In [4]:
total_data = pd.DataFrame([])

In [5]:
data = pd.read_csv("./input/combined_data_1.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,1)
#print "Counter",1
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape
#print mem_usage(data), mem_usage(total_data), " ", mem_usage(temp_data[0])

Before (0, 0)
After (24053764, 4)


In [6]:
data = pd.read_csv("./input/combined_data_2.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape
#print mem_usage(data), mem_usage(total_data), " ", mem_usage(temp_data[0])

Counter 9209
Before (24053764, 4)
After (51031355, 4)


In [11]:
print "After",total_data.shape

After (73632984, 4)


In [9]:
data = pd.read_csv("./input/combined_data_3.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape

Counter 13365
Before (51031355, 4)
After (73632984, 4)


In [ ]:
total_data.shape

In [7]:
data = pd.read_csv("./input/combined_data_4.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape


In [15]:
#del data
sample_df = total_data[:4000]

In [16]:
sample_df.head()

,Cust_Id,Rating,date,movie_id
0,1488844,3.0,2005-09-06,1.0
1,822109,5.0,2005-05-13,1.0
2,885013,4.0,2005-10-19,1.0
3,30878,4.0,2005-12-26,1.0
4,823519,3.0,2004-05-03,1.0


In [17]:
temp = pd.pivot_table(sample_df,index='Cust_Id', columns='movie_id')
# temp = pd.pivot_table(sample_df,'movie_id','Cust_Id'

### Create models as a dictionary below

In [12]:
coclustering = CoClustering()
knnbaseline = KNNBaseline()
knnwithmeans = KNNWithMeans()
svd = SVD()
svdpp = SVDpp()
modelList = {"coclustering" : coclustering, "knnbaseline" : knnbaseline, "knnwithmeans" : knnwithmeans, "svd" :svd, "svdpp" :svdpp}

In [39]:
reader = Reader()
                                                                                                                                                                                                                                                                                                                                                
# get just top 100K rows for faster run time
trainData = Dataset.load_from_df(total_data[['Cust_Id', 'movie_id', 'Rating']].sample(n=10000000), reader)
#data.split(n_folds=3)
#trainset = data.build_full_trainset()

In [40]:
trainData.split(n_folds=3)
trainset = trainData.build_full_trainset()

In [41]:
# Start of Probe Data manipulation 
probe = pd.read_csv("./input/probe.txt",  names = ['Cust_Id'])
# temp_data = processDataFile(probe,1)
probe_movie = pd.DataFrame([])
Cust_Id = []
movie_id = []
currentMovie =0
index = 0

for row in probe['Cust_Id']:
    if row[len(row)-1] == ":":
        currentMovie = int(row[:-1])
        #print currentMovie
        continue
    Cust_Id.append(row)
    movie_id.append(currentMovie)
    #print index,row
    index +=1
probe_movie['Cust_Id'] = Cust_Id
probe_movie['movie_id'] = movie_id
#print probe_movie[:30]
# test = total_data.loc[(total_data.Cust_Id.isin(probe_movie.Cust_Id)) & total_data.movie_id.isin(probe_movie.movie_id)]
merged_probe_set = pd.merge(probe_movie,total_data,on=['Cust_Id','movie_id'])
#print merged_probe_set[:30]
# End of Probe Data manipulation 

In [42]:
print len(probe)

1425333


In [44]:
def evaluateTrainAndTestModel(modelList, testData, predictions):
    # Evaluate model
    for model in modelList:
        print "Testing model: ", model
        try:
            evaluate(modelList[model], trainData, measures=['RMSE', 'MAE'])

            # Train model
            modelList[model].train(trainset)

            # Predict Movie ratings and compute error
            cust_id = [];
            pred = [];
            index =1;
            mae_sum = 0;
            mse_sum = 0;
            mse_sum_round = 0;
            rmse_round = 0;
            rmse = 0;
            n = len(merged_probe_set)
            for index,row in testData.iterrows():
                user = row['Cust_Id']
                currentMovie = row['movie_id']
                pred = modelList[model].predict(user, currentMovie, r_ui=row['Rating'], verbose=False)
                predictions[model][index]= pred.est
                diff = row['Rating'] - pred.est
                diffRound = row['Rating'] - round(pred.est)
                mse_sum += diff**2
                mse_sum_round += diffRound**2
                #print(diff)
            mse = mse_sum/n
            rmse = (mse_sum/n)**0.5
            rmse_round = (mse_sum_round/n)**0.5

            # TODO: Try using model.test
            #pred = model.test(testData['Cust_Id'])
            #rmse(pred)
            #dump.dump('./dump_SVD', pred, model)
            #print testData[:20]
            print "MSE: ", mse, " RMSE: ", rmse , " RMSE Roundup:  ", rmse_round
        except MemoryError:
            print "!!! Memory Error occured !!!"
            print "Continuing with next model"
    return predictions

In [45]:
predictions = pd.DataFrame(0, index=np.arange(len(merged_probe_set)), columns=modelList.keys())
evaluateTrainAndTestModel(modelList, merged_probe_set, predictions)


Testing model:  coclustering
Evaluating RMSE, MAE of algorithm CoClustering.

------------
Fold 1
RMSE: 0.9854
MAE:  0.7670
------------
Fold 2
RMSE: 0.9840
MAE:  0.7657
------------
Fold 3
RMSE: 0.9839
MAE:  0.7658
------------
------------
Mean RMSE: 0.9845
Mean MAE : 0.7662
------------
------------
    Cust_Id movie_id  Rating        date
0     30878        1     4.0  2005-12-26
1   2647871        1     4.0  2005-12-30
2   1283744        1     3.0  2004-04-17
3   2488120        1     5.0  2005-09-20
4    317050        1     5.0  2005-11-15
5   1904905        1     4.0  2005-05-13
6   1989766        1     4.0  2005-07-08
7     14756        1     4.0  2005-12-27
8   1027056        1     3.0  2005-12-03
9   1149588        1     4.0  2005-12-13
10  1394012        1     5.0  2005-12-19
11  1406595        1     4.0  2005-08-27
12  2529547        1     5.0  2005-11-18
13  1682104        1     4.0  2005-08-30
14  2625019        1     3.0  2005-09-12
15  2603381        1     5.0  2005-11-29

In [46]:
print predictions

        coclustering  knnbaseline  svd  knnwithmeans  svdpp
0                  3            0    0             0      0
1                  3            0    0             0      0
2                  3            0    0             0      0
3                  4            0    0             0      0
4                  4            0    0             0      0
5                  4            0    0             0      0
6                  3            0    0             0      0
7                  3            0    0             0      0
8                  2            0    0             0      0
9                  3            0    0             0      0
10                 3            0    0             0      0
11                 3            0    0             0      0
12                 3            0    0             0      0
13                 4            0    0             0      0
14                 3            0    0             0      0
15                 4            0    0  